In [7]:
pip install OpenDartReader

  Using cached OpenDartReader-0.1.6-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\wkdwoo\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [1]:
import OpenDartReader
import pandas as pd
api_key = '85c2367fdad752f376b3947ccdf9998092765f0e'
dart = OpenDartReader(api_key)

In [2]:
dart

In [3]:
fs_2020 = dart.finstate_all(corp='005930', bsns_year='2020', fs_div='CFS', reprt_code=11011)
fs_2021_3Q = dart.finstate_all(corp='005930', bsns_year='2021', fs_div='CFS', reprt_code=11014)

In [4]:
fs_2021_3Q.loc[fs_2021_3Q['sj_div'].isin(['BS']) & fs_2021_3Q['account_id'].isin(['ifrs-full_Equity']), ]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
51,20211115001965,11014,2021,00126380,BS,재무상태표,ifrs-full_Equity,자본총계,-,제 53 기 3분기말,296766118000000,제 52 기말,275948016000000,54,NaN,NaN,NaN,NaN


In [5]:
fs_2021_3Q.loc[fs_2021_3Q['sj_div'].isin(['BS']) & fs_2021_3Q['account_id'].isin(['ifrs-full_Liabilities']), ]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
41,20211115001965,11014,2021,00126380,BS,재무상태표,ifrs-full_Liabilities,부채총계,-,제 53 기 3분기말,113654600000000,제 52 기말,102287702000000,43,NaN,NaN,NaN,NaN


In [6]:
fs_2021_3Q.loc[fs_2021_3Q['sj_div'].isin(['IS']) & fs_2021_3Q['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), ]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
67,20211115001965,11014,2021,00126380,IS,손익계산서,ifrs-full_ProfitLossAttributableToOwnersOfParent,지배기업의 소유주에게 귀속되는 당기순이익(손실),-,제 53 기 3분기,12057207000000,NaN,NaN,15,28600669000000,제 52 기 3분기,9266814000000,19645377000000


In [8]:
# 자본과 부채는 재무상태표에서 당기금액('thstrm_amount') 값을 가져오면 됨
equity = int(fs_2021_3Q.loc[fs_2021_3Q['sj_div'].isin(['BS']) & fs_2021_3Q['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount'].replace(",", "")) # 당기자본(자본총계)
liability = int(fs_2021_3Q.loc[fs_2021_3Q['sj_div'].isin(['BS']) & fs_2021_3Q['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",", "")) # 당기부채(부채총계)
assets = equity + liability # 자산총계

In [9]:
equity

296766118000000

In [10]:
liability

113654600000000

In [11]:
assets

410420718000000

In [17]:
# 2020 4분기 ~ 2021 3분기까지의 당기순이익의 합을 구하려면 2020년 4분기 당기순이익과 2021년 1분기 ~ 3분기 당기순이익의 합을 알아여함
# 2021년 1분기 ~ 3분기 당기순이익의 합은 2021년 3분기 손익계산서에서 'thstrm_add_amount' 값을 가져오면 되고
# 2020년 4분기 당기순이익은 2020년 전체 당기순이익에서 2020년 1분기 ~ 3분기 당기순이익의 합을 빼서 구할 수 있음
profit_2020_3Q = int(fs_2021_3Q.loc[fs_2021_3Q['sj_div'].isin(['IS']) & fs_2021_3Q['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'frmtrm_add_amount'].replace(",", "")) # 당기순이익
profit_2020 = int(fs_2020.loc[fs_2020['sj_div'].isin(['IS']) & fs_2020['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_amount'].replace(",", "")) # 당기순이익
profit_2021_3Q = int(fs_2021_3Q.loc[fs_2021_3Q['sj_div'].isin(['IS']) & fs_2021_3Q['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_add_amount'].replace(",", "")) # 당기순이익
profit = (profit_2020-profit_2020_3Q) + profit_2021_3Q
profit

35046138000000

In [34]:
small = dart.report('005930', '소액주주', 2021, reprt_code=11014)
small

,rcept_no,corp_cls,corp_code,corp_name,se,shrholdr_co,shrholdr_tot_co,shrholdr_rate,hold_stock_co,stock_tot_co,hold_stock_rate
0,20211115001965,Y,00126380,삼성전자,소액주주,"5,188,804","5,188,924",99.99%,"3,834,408,221","5,969,782,550",64.23%


In [19]:
stock_tot_co = int(small['stock_tot_co'].str.replace(',', ''))
stock_tot_co

5969782550

In [20]:
from marcap import marcap_data

In [23]:
# 삼성전자
df_005930 = marcap_data('2022-01-28', code='005930')
df_005930 = df_005930.assign(Amount=df_005930['Amount'].astype('int64'),
                             Marcap=df_005930['Marcap'].astype('int64'))
df_005930

,Code,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId,Rank
Date,,,,,,,,,,,,,,,,,
2022-01-28,005930,삼성전자,KOSPI,NaN,73300,1,2000,2.81,71300,73700,71200,21367447,1552585967650,437585060915000,5969782550,STK,1


In [24]:
df_005930['Stocks'].iat[0]

5969782550

In [25]:
# 시가총액/종가 로 계산하면 맞게 나옴
stocks_005930 = int(df_005930['Marcap']/df_005930['Close'])
stocks_005930

5969782550

In [26]:
# 삼성전자우
df_005935 = marcap_data('2021-01-21', code='005935')
df_005935 = df_005935.assign(Amount=df_005935['Amount'].astype('int64'),
                             Marcap=df_005935['Marcap'].astype('int64'))
df_005935

,Code,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId,Rank
Date,,,,,,,,,,,,,,,,,
2021-01-21,005935,삼성전자우,KOSPI,NaN,77600,1,800,1.04,77500,77800,76800,2512634,194360911300,63856007920000,822886700,STK,4


In [27]:
df_005935['Stocks'].iat[0]

822886700

In [28]:
stocks_005935 = int(df_005935['Marcap']/df_005935['Close'])
stocks_005935

822886700

In [29]:
total_stocks = stocks_005930 + stocks_005935
total_stocks

6792669250

In [30]:
EPS = profit/total_stocks
PER = df_005930['Close'].iat[0]/EPS
print('EPS: {}, PER: {}'.format(int(round(EPS)), round(PER, 2)))

EPS: 5159, PER: 14.21


In [31]:
BPS = equity/total_stocks
PBR = df_005930['Close'].iat[0]/BPS
print('BPS: {}, PBR: {}'.format(int(round(BPS)), round(PBR, 2)))

BPS: 43689, PBR: 1.68


In [32]:
ROE = PBR/PER
ROA = profit/assets
print('ROE: {}%, ROA: {}%'.format(round(ROE*100, 2), round(ROA*100, 2)))

ROE: 11.81%, ROA: 8.54%
